In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

In [7]:
mpl.matplotlib_fname()
font_dirs = ['D:\\Software\\Anaconda\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf' ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
mpl.font_manager._rebuild()
result = font_manager.findfont("Rounded Mplus 1c")
print(result)
mpl.rc('font', family='Rounded Mplus 1c')

D:\Anaconda\Lib\site-packages\matplotlib\mpl-data\fonts\ttf\MPLUSRounded1c-Regular.ttf


In [8]:
# df_merged = pd.read_csv("E:\\Projects\\KAIHO\\Data\\Kaiho_merged_data.csv")
df_merged = pd.read_csv("Kaiho_merged_data.csv")

In [9]:
list_col = list(df_merged.columns)
list_fil = list(filter(lambda col :  df_merged[col].nunique() > 1 ,list_col))
df_fil = df_merged[list_fil]
df_fil['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額'] = df_fil['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額'].apply(lambda x: x.replace(',', '')).astype('float64')
df_fil.drop(columns = ['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額.1'],inplace= True)

<ipython-input-9-af902f39b4fc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額'] = df_fil['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額'].apply(lambda x: x.replace(',', '')).astype('float64')


In [10]:
for col in list(df_fil.columns):
 (df_fil[col].dtype ==  'object') and (df_fil[col].fillna(value='unknown',inplace = True)) or (df_fil[col].fillna(value=0, inplace = True))

In [11]:
df_fil.isnull().sum()

Vị trí_ロケーション                                           0
Ngày sản xuất_生産日                                       0
Tháng sản xuất_生産月                                      0
Năm sản xuất_生産年                                        0
Nv sản xuất_生産担当者                                       0
Số nhập kho_入庫番号                                        0
Nguồn nhập hàng_仕入先                                     0
Mã loại phụ tùng_商品コード                                  0
Tên phụ tùng_商品名                                        0
Số nhập kho(phụ tùng)_入庫番号 (部品)                         0
Tên xe_車名                                               0
Model xe_車輌型式                                           0
Engine model_エンジン型式                                     0
Tổng giá nhập_仕入合計                                      0
Số phút công_標準工数                                       0
Kết quả_生産高                                             0
Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額    0
Ước tính giá b

In [12]:
df_fil.groupby('Tên phụ tùng_商品名')['Tên phụ tùng_商品名'].count()

Tên phụ tùng_商品名
エンジン                14926
ノーズカット（個別）           3524
フェイスカット                 8
フロントアクスル（個別）          158
ミッション（個別）               2
リアアクスル（個別）            109
リアホーシング（デフ無）（個別）        1
外装コンプリート               13
ＣＫＤ／コンプリート           1207
ＣＫＤ／コンプリート（バイヤー）       22
Ｆアクスル（生/Ｄ）             69
Ｒアクスル（生/Ｄ）            389
Name: Tên phụ tùng_商品名, dtype: int64

In [13]:
df_fil.groupby('Tên phụ tùng_商品名')['Thành tích bán ra_販売実績'].sum()

Tên phụ tùng_商品名
エンジン                549620961.5
ノーズカット（個別）           37470500.0
フェイスカット                162000.0
フロントアクスル（個別）          1353000.0
ミッション（個別）               10000.0
リアアクスル（個別）             787000.0
リアホーシング（デフ無）（個別）         8000.0
外装コンプリート              1087450.0
ＣＫＤ／コンプリート          166435819.0
ＣＫＤ／コンプリート（バイヤー）      1710000.0
Ｆアクスル（生/Ｄ）             351500.0
Ｒアクスル（生/Ｄ）            2472500.0
Name: Thành tích bán ra_販売実績, dtype: float64

In [14]:
df_best_phutung = df_fil[(df_fil['Tên phụ tùng_商品名']=='エンジン') | (df_fil['Tên phụ tùng_商品名']=='ノーズカット（個別）') |(df_fil['Tên phụ tùng_商品名']== 'ＣＫＤ／コンプリート')]
df_best_phutung

,Vị trí_ロケーション,Ngày sản xuất_生産日,Tháng sản xuất_生産月,Năm sản xuất_生産年,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),...,Tháng mua hàng_仕入月,Năm mua_仕入年,Chi tiết xếp hạng_ランク詳細,Số phiếu_伝票番号,Mã nhà cung cấp_仕入先コード,Số body_車体番号,CONTAINER_CONTAINER,Tên khách hàng_売上先名,D/G_Ｄ／Ｇ,Đang tải nguồn_積込元
0,会宝産業本社,17,11,2015,山下大輝,'0000000302676,西田 敏勝,'110010,ノーズカット（個別）,'0000007988203,...,12.0,2012.0,unknown,39332,990010.0,'1001019,unknown,"一番オート OOO ""LIGHT VL""",G,会宝産業本社
1,会宝産業本社,3,2,2015,中嶋 修,'0000000303771,ｴｻﾞｼ自動車,'100000,エンジン,'0000007739581,...,1.0,2015.0,unknown,24925,990000.0,'6062039,40HC,木戸クンク・ジャイブ（ＥＫ３）,G,会宝産業本社
2,会宝産業本社,3,2,2015,楊 志毅,'0000000303771,ｴｻﾞｼ自動車,'110010,ノーズカット（個別）,'0000007712751,...,1.0,2015.0,unknown,24925,990000.0,'6062039,40HC,木戸クンク・ジャイブ（ＥＫ３）,G,会宝産業本社
3,会宝産業本社,5,2,2015,西川航平,'0000000314958,ＪＵ石川,'110010,ノーズカット（個別）,'0000007732276,...,1.0,2015.0,@,24945,319800.0,'2010272,40HC,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
4,会宝産業本社,25,2,2015,中里太明,'0000000319165,会宝パーツサービス㈲福井店 アップガレージ,'110010,ノーズカット（個別）,'0000007742703,...,2.0,2015.0,unknown,25365,208750.0,'072307,40HC,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20401,会宝産業本社,26,9,2013,中嶋 修,'0000000314082,ＧＭＳコーポレーシ(ｶｰｸﾘｴｲﾄ金沢),'100000,エンジン,'0000007353787,...,9.0,2013.0,unknown,19028,200460.0,'124900,20,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
20402,会宝産業本社,30,9,2013,菊池忠義,'0000000314150,㈱日産プリンス東大通店,'100000,エンジン,'0000007356752,...,9.0,2013.0,unknown,19028,511610.0,'201247,20,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
20403,会宝産業本社,28,9,2013,坂本光司,'0000000314364,JU岐阜 羽島オークション,'100000,エンジン,'0000007355332,...,9.0,2013.0,@,19028,310490.0,'0044142,20,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
20405,会宝産業本社,28,2,2012,菊池忠義,'0000000287252,Ｋ・Ｓ・Ｋ,'110010,ノーズカット（個別）,'0000005894104,...,2.0,2012.0,以前のバラし分,20406,208900.0,'7063020,20,FADEEV EVGENY,G,会宝産業本社


In [15]:
cat_col =  list(df_best_phutung.select_dtypes(include='object'))
df_cat  = df_best_phutung[cat_col]
df_cat.isnull().sum()

Vị trí_ロケーション                      0
Nv sản xuất_生産担当者                  0
Số nhập kho_入庫番号                   0
Nguồn nhập hàng_仕入先                0
Mã loại phụ tùng_商品コード             0
Tên phụ tùng_商品名                   0
Số nhập kho(phụ tùng)_入庫番号 (部品)    0
Tên xe_車名                          0
Model xe_車輌型式                      0
Engine model_エンジン型式                0
Khách hàng_仕向先                     0
Truyền động 1_駆動1                  0
TM_TM                              0
Truyền động 2_駆動2                  0
Chi tiết_詳細                        0
Rank_ランク                           0
Loại xe_車種                         0
Nhà sản xuất_メーカー                  0
Chi tiết xếp hạng_ランク詳細            0
Số body_車体番号                       0
CONTAINER_CONTAINER                0
Tên khách hàng_売上先名                0
D/G_Ｄ／Ｇ                            0
Đang tải nguồn_積込元                 0
dtype: int64

In [16]:
num_col = list(df_best_phutung.select_dtypes(include=['int64','float64']))
df_num = df_best_phutung[num_col]
df_num.isnull().sum()

Ngày sản xuất_生産日                                       0
Tháng sản xuất_生産月                                      0
Năm sản xuất_生産年                                        0
Tổng giá nhập_仕入合計                                      0
Số phút công_標準工数                                       0
Kết quả_生産高                                             0
Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額    0
Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額     0
Số lượng bán ra_販売数量                                    0
Thành tích bán ra_販売実績                                  0
Số km đã chạy_走行距離                                      0
Năm sản xuất_年式                                         0
Số chỉ định loại_型式指定番号                                 0
Số phân loại danh mục_類別区分番号                            0
Đơn giá mua_仕入単価                                        0
Giờ làm việc_工数                                         0
Đơn giá bán hàng_売上単価                                   0
Sản lượng bán 

In [17]:
df_target = df_best_phutung['Thành tích bán ra_販売実績']
df_target

0        12000.0
1        30000.0
2        10000.0
3        12000.0
4        10000.0
          ...   
20401    25000.0
20402    20000.0
20403    30000.0
20405    12000.0
20423    12000.0
Name: Thành tích bán ra_販売実績, Length: 19657, dtype: float64

In [18]:
le = LabelEncoder()
df_cat_encoded = df_cat.apply(lambda col: le.fit_transform(col))

In [19]:
df_encode = pd.merge(df_cat_encoded, df_num, on = df_cat_encoded.index, how='outer')
df_encode.drop(columns = ['key_0'], inplace=True)
df_encode

,Vị trí_ロケーション,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,...,Đơn giá bán hàng_売上単価,Sản lượng bán ra_売上金額,Ngày bán hàng_売上日,Tháng bán hàng_売上月,Năm bán hàng_売上年,Ngày mua_仕入日,Tháng mua hàng_仕入月,Năm mua_仕入年,Số phiếu_伝票番号,Mã nhà cung cấp_仕入先コード
0,1,22,3831,2444,2,1,15680,269,365,383,...,12000.0,12000,30,3,2016,22.0,12.0,2012.0,39332,990010.0
1,1,5,3943,2757,1,0,14310,210,72,153,...,30000.0,30000,19,2,2015,31.0,1.0,2015.0,24925,990000.0
2,1,30,3943,2757,2,1,14147,210,72,153,...,10000.0,10000,19,2,2015,31.0,1.0,2015.0,24925,990000.0
3,1,43,5443,2707,2,1,14269,286,1054,231,...,12000.0,12000,23,2,2015,15.0,1.0,2015.0,24945,319800.0
4,1,9,6078,803,2,1,14337,295,4,472,...,10000.0,10000,24,3,2015,23.0,2.0,2015.0,25365,208750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19652,1,5,5292,2702,1,0,11032,301,1244,418,...,0.0,0,8,10,2013,21.0,9.0,2013.0,19028,200460.0
19653,1,41,5303,518,1,0,11068,412,1123,286,...,0.0,0,8,10,2013,26.0,9.0,2013.0,19028,511610.0
19654,1,19,5357,82,1,0,11052,53,893,23,...,0.0,0,8,10,2013,25.0,9.0,2013.0,19028,310490.0
19655,1,41,1717,2709,2,1,7625,211,1196,70,...,0.0,0,6,2,2014,24.0,2.0,2012.0,20406,208900.0


In [20]:
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

In [21]:
df_scale = pd.DataFrame(ss.fit_transform(df_encode),columns = df_encode.columns)

In [22]:
df_scale.drop(columns=['Thành tích bán ra_販売実績','Đơn giá bán hàng_売上単価','Sản lượng bán ra_売上金額'],inplace=True)

In [37]:
x_train, x_test, y_train, y_test = train_test_split(df_scale, df_target, test_size=0.2,random_state=42)

In [38]:
names = ["Gradient_Boosting", "Decision_Tree", "Extra_Trees", "Random_Forest","LGBM","XGBoost"]
models = [
    GradientBoostingRegressor(),
    DecisionTreeRegressor(max_depth=5),
    ExtraTreesRegressor(),
    RandomForestRegressor(),
    LGBMRegressor(),
    XGBRegressor()]

In [39]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[30967.47857737 28799.12177555 24211.77539813 ... 10728.60645418
 24942.46975479 35893.21886494]
[31767.76282478 26276.18166008 26276.18166008 ... 10297.9568672
 26276.18166008 36118.78684211]
[29940. 32180. 24300. ...  8540. 24760. 35290.]
[31270. 30670. 23930. ... 10720. 23685. 35420.]
[30343.2596382  27674.10171044 22706.33186613 ... 10818.82298534
 24444.67759476 35536.1084823 ]
[31251.096  30522.234  25282.771  ...  8183.3203 23286.576  35933.742 ]


In [40]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.937,8702.709,13.778
1,Decision_Tree,0.909,10514.196,17.117
2,Extra_Trees,0.951,7671.400,10.815
3,Random_Forest,0.940,8554.399,11.394
4,LGBM,0.948,7958.781,12.605
5,XGBoost,0.944,8237.863,12.307


In [41]:
shap_list = ['Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額',
            'Mã loại phụ tùng_商品コード',
            'Tổng giá nhập_仕入合計',
            'Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額',
            'Tên khách hàng_売上先名',
            'Engine model_エンジン型式',
            'Số phút công_標準工数',
            'Số nhập kho_入庫番号',
            'Khách hàng_仕向先']

In [43]:
df_shap = df_scale[shap_list]
df_shap

,Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額,Mã loại phụ tùng_商品コード,Tổng giá nhập_仕入合計,Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額,Tên khách hàng_売上先名,Engine model_エンジン型式,Số phút công_標準工数,Số nhập kho_入庫番号,Khách hàng_仕向先
0,-0.626250,1.852361,-0.758469,-0.667362,1.046165,1.168644,-0.920594,-0.994754,0.235536
1,-0.274946,-0.247516,0.053766,0.133472,1.244081,-0.329707,0.466135,-0.973251,1.108467
2,-0.626250,1.852361,-0.596022,-0.347029,1.244081,-0.329707,-0.920594,-0.973251,1.108467
3,-0.626250,1.852361,-0.732153,-0.667362,0.980193,0.178429,-0.920594,-0.685268,0.904783
4,-0.626250,1.852361,-0.732153,-0.667362,0.980193,1.748440,-0.920594,-0.563355,0.904783
...,...,...,...,...,...,...,...,...,...
19652,-0.274946,-0.247516,0.053766,0.133472,0.980193,1.396654,1.506182,-0.714258,0.904783
19653,-0.410063,-0.247516,-0.108681,-0.026695,0.980193,0.536731,0.466135,-0.712146,0.904783
19654,-0.139829,-0.247516,0.216213,0.293639,0.980193,-1.176601,-0.227230,-0.701779,0.904783
19655,-0.626250,1.852361,-0.719360,-0.667362,-0.932995,-0.870416,-0.920594,-1.400619,-0.986566


In [56]:
x_train, x_test, y_train, y_test = train_test_split(df_shap, df_target, test_size=0.2,random_state=42)

In [57]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[29907.54067693 28018.58704953 24457.53931729 ... 10873.32982712
 25433.4833822  35955.41220668]
[31767.76282478 26276.18166008 26276.18166008 ... 10326.95554305
 26276.18166008 36118.78684211]
[29920. 28580. 24840. ...  7060. 24990. 35150.]
[28930. 29030. 24650. ...  7840. 25120. 35300.]
[29414.13471896 27621.62208066 24156.9957888  ... 10580.86460316
 24667.63441669 35446.34314487]
[32660.986 28748.535 25066.998 ...  8977.389 25956.77  36605.945]


In [58]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.938,8697.266,14.242
1,Decision_Tree,0.909,10475.233,17.137
2,Extra_Trees,0.945,8142.879,11.338
3,Random_Forest,0.939,8614.135,11.320
4,LGBM,0.941,8418.432,12.965
5,XGBoost,0.944,8201.795,12.151


In [65]:
df_select = df_scale[select_list]
df_select

,Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,Truyền động 1_駆動1,D/G_Ｄ／Ｇ,Rank_ランク,Loại xe_車種,Ngày bán hàng_売上日,Tháng bán hàng_売上月,Năm bán hàng_売上年
0,0.144182,-0.759955,1.168644,-0.489430,0.273187,2.071244,-0.584226,1.143551,-1.096938,0.188656
1,-0.339634,-1.406283,-0.329707,-0.489430,0.273187,-0.862796,-0.584226,-0.231650,-1.402824,-0.139319
2,-0.339634,-1.406283,-0.329707,-0.489430,0.273187,-0.862796,-0.584226,-0.231650,-1.402824,-0.139319
3,0.283586,0.759907,0.178429,-0.489430,0.273187,-0.862796,-0.584226,0.268423,-1.402824,-0.139319
4,0.357389,-1.556284,1.748440,-0.489430,0.273187,-0.862796,1.143361,0.393442,-1.096938,-0.139319
...,...,...,...,...,...,...,...,...,...,...
19652,0.406590,1.179027,1.396654,-0.489430,0.273187,0.359721,-0.584226,-1.606851,1.044264,-0.795269
19653,1.316820,0.912113,0.536731,1.714125,0.273187,1.093231,1.143361,-1.606851,1.044264,-0.795269
19654,-1.627075,0.404757,-1.176601,-0.489430,0.273187,1.093231,-0.584226,-1.606851,1.044264,-0.795269
19655,-0.331433,1.073144,-0.870416,-0.489430,0.273187,-0.862796,-0.584226,-1.856888,-1.402824,-0.467294


In [66]:
x_train, x_test, y_train, y_test = train_test_split(df_select, df_target, test_size=0.2,random_state=42)

In [67]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[26881.26908993 27433.73738804 21772.21972787 ... 23409.43202336
 26775.26253546 30834.66797534]
[27401.63526781 27401.63526781 27401.63526781 ... 27401.63526781
 27401.63526781 27401.63526781]
[30000.   29985.   23950.   ... 36636.45 23470.   10000.  ]
[33000.         27195.         24150.         ... 37160.
 21645.         19253.33333333]
[29438.70223021 24680.89799065 19550.27881506 ... 24091.30665091
 25769.45982683 27859.96688755]
[34102.91  25034.377 22540.744 ... 25293.732 25874.176 29584.672]


In [68]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.507,24445.439,60.505
1,Decision_Tree,0.366,27699.942,67.295
2,Extra_Trees,0.608,21792.012,51.298
3,Random_Forest,0.614,21608.967,51.091
4,LGBM,0.603,21912.167,53.960
5,XGBoost,0.637,20966.160,49.847


# Phu tung エンジン

In [69]:
df_エンジン = df_fil[(df_fil['Tên phụ tùng_商品名']=='エンジン')]

In [70]:
cat_col =  list(df_エンジン.select_dtypes(include='object'))
df_cat  = df_エンジン[cat_col]
df_cat.isnull().sum()

Vị trí_ロケーション                      0
Nv sản xuất_生産担当者                  0
Số nhập kho_入庫番号                   0
Nguồn nhập hàng_仕入先                0
Mã loại phụ tùng_商品コード             0
Tên phụ tùng_商品名                   0
Số nhập kho(phụ tùng)_入庫番号 (部品)    0
Tên xe_車名                          0
Model xe_車輌型式                      0
Engine model_エンジン型式                0
Khách hàng_仕向先                     0
Truyền động 1_駆動1                  0
TM_TM                              0
Truyền động 2_駆動2                  0
Chi tiết_詳細                        0
Rank_ランク                           0
Loại xe_車種                         0
Nhà sản xuất_メーカー                  0
Chi tiết xếp hạng_ランク詳細            0
Số body_車体番号                       0
CONTAINER_CONTAINER                0
Tên khách hàng_売上先名                0
D/G_Ｄ／Ｇ                            0
Đang tải nguồn_積込元                 0
dtype: int64

In [71]:
num_col = list(df_エンジン.select_dtypes(include=['int64','float64']))
df_num = df_エンジン[num_col]
df_num.isnull().sum()

Ngày sản xuất_生産日                                       0
Tháng sản xuất_生産月                                      0
Năm sản xuất_生産年                                        0
Tổng giá nhập_仕入合計                                      0
Số phút công_標準工数                                       0
Kết quả_生産高                                             0
Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額    0
Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額     0
Số lượng bán ra_販売数量                                    0
Thành tích bán ra_販売実績                                  0
Số km đã chạy_走行距離                                      0
Năm sản xuất_年式                                         0
Số chỉ định loại_型式指定番号                                 0
Số phân loại danh mục_類別区分番号                            0
Đơn giá mua_仕入単価                                        0
Giờ làm việc_工数                                         0
Đơn giá bán hàng_売上単価                                   0
Sản lượng bán 

In [72]:
df_target = df_エンジン['Thành tích bán ra_販売実績']
df_target

1        30000.0
6        25000.0
8        45000.0
10       30000.0
11       85000.0
          ...   
20384    15000.0
20400    25000.0
20401    25000.0
20402    20000.0
20403    30000.0
Name: Thành tích bán ra_販売実績, Length: 14926, dtype: float64

In [73]:
le = LabelEncoder()
df_cat_encoded = df_cat.apply(lambda col: le.fit_transform(col))
df_encode = pd.merge(df_cat_encoded, df_num, on = df_cat_encoded.index, how='outer')
df_encode.drop(columns = ['key_0'], inplace=True)
df_encode

,Vị trí_ロケーション,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,...,Đơn giá bán hàng_売上単価,Sản lượng bán ra_売上金額,Ngày bán hàng_売上日,Tháng bán hàng_売上月,Năm bán hàng_売上年,Ngày mua_仕入日,Tháng mua hàng_仕入月,Năm mua_仕入年,Số phiếu_伝票番号,Mã nhà cung cấp_仕入先コード
0,0,4,3209,2344,0,0,10770,194,62,141,...,30000.0,30000,19,2,2015,31.0,1.0,2015.0,24925,990000.0
1,0,20,5018,1388,0,0,11083,243,797,104,...,25000.0,25000,30,4,2015,15.0,4.0,2015.0,25896,518170.0
2,0,20,5277,2302,0,0,10736,286,314,125,...,45000.0,45000,23,2,2015,5.0,4.0,2014.0,24945,319800.0
3,0,20,5449,263,0,0,11408,194,62,141,...,30000.0,30000,31,8,2015,3.0,4.0,2014.0,37373,304510.0
4,0,17,5450,377,0,0,10690,156,1405,28,...,85000.0,85000,14,1,2015,20.0,12.0,2014.0,24515,701400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14921,0,4,12139,1618,0,0,2358,21,545,332,...,0.0,0,11,4,2019,30.0,3.0,2019.0,52628,990010.0
14922,0,17,4273,30,0,0,8126,361,1330,389,...,0.0,0,8,10,2013,24.0,9.0,2013.0,19028,704400.0
14923,0,4,4322,2297,0,0,8130,280,1152,389,...,0.0,0,8,10,2013,21.0,9.0,2013.0,19028,200460.0
14924,0,36,4333,468,0,0,8166,384,1037,268,...,0.0,0,8,10,2013,26.0,9.0,2013.0,19028,511610.0


In [74]:
ss = StandardScaler()
df_scale = pd.DataFrame(ss.fit_transform(df_encode),columns = df_encode.columns)
df_scale.drop(columns=['Thành tích bán ra_販売実績','Đơn giá bán hàng_売上単価','Sản lượng bán ra_売上金額'],inplace=True)
x_train, x_test, y_train, y_test = train_test_split(df_scale, df_target, test_size=0.2,random_state=42)

In [75]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[24393.08643972 64083.67338363 32549.37844529 ... 26021.55866017
 32363.48635322 29688.17677048]
[26231.12992701 59372.47342995 27337.83397683 ... 27337.83397683
 30602.39707016 26231.12992701]
[27485. 59450. 30140. ... 26580. 34495. 24730.]
[27195.   68660.28 29750.   ... 25960.   33410.   26290.  ]
[24693.21796079 68701.09853535 32717.73851974 ... 25919.98894948
 37069.53651894 29025.47236984]
[22768.959 69872.695 34720.26  ... 23730.979 39324.805 31973.852]


In [76]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.874,7770.840,13.654
1,Decision_Tree,0.815,9415.431,16.764
2,Extra_Trees,0.886,7398.560,11.878
3,Random_Forest,0.874,7759.864,12.324
4,LGBM,0.885,7426.279,12.110
5,XGBoost,0.883,7498.687,11.940


In [78]:
df_shap = df_scale[shap_list]
df_shap

,Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額,Mã loại phụ tùng_商品コード,Tổng giá nhập_仕入合計,Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額,Tên khách hàng_売上先名,Engine model_エンジン型式,Số phút công_標準工数,Số nhập kho_入庫番号,Khách hàng_仕向先
0,-0.354034,0.0,0.096008,0.093054,1.119903,-0.352448,0.050981,-0.987174,1.087427
1,-0.354034,0.0,-0.422123,-0.950239,1.119903,-0.613234,1.099833,-0.567332,1.087427
2,-0.121480,0.0,0.372345,0.301713,0.983141,-0.465220,-0.997871,-0.507222,1.196601
3,-0.121480,0.0,0.664506,-0.950239,-0.042570,-0.352448,0.050981,-0.467303,0.687121
4,1.273844,0.0,2.030366,1.553665,0.983141,-1.148903,-0.997871,-0.467071,1.196601
...,...,...,...,...,...,...,...,...,...
14921,-0.819142,0.0,-1.103447,-0.950239,-1.410184,0.993773,-0.997871,1.085346,-1.496363
14922,-0.354034,0.0,0.096008,0.093054,0.983141,1.395525,0.050981,-0.740235,0.832687
14923,-0.354034,0.0,0.096008,0.093054,0.983141,1.395525,1.624258,-0.728863,0.832687
14924,-0.586588,0.0,-0.180329,-0.115604,0.983141,0.542683,0.050981,-0.726310,0.832687


In [79]:
x_train, x_test, y_train, y_test = train_test_split(df_shap, df_target, test_size=0.2,random_state=42)

In [80]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[24218.54211242 63304.61428334 33218.59290248 ... 25221.8319642
 33579.73977411 27562.25067186]
[26236.27947598 56072.5234375  28167.72936803 ... 28167.72936803
 30602.39707016 26236.27947598]
[26350.   60862.55 30210.   ... 25400.   42090.   25040.  ]
[26350.   66572.55 28330.   ... 25180.   42700.   27510.  ]
[24531.27241511 61647.30043573 35344.7999167  ... 25999.09222334
 39052.80370026 27429.91327872]
[25704.744 65507.508 32773.902 ... 25948.332 41233.74  30239.281]


In [81]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.869,7929.966,14.267
1,Decision_Tree,0.827,9109.357,16.151
2,Extra_Trees,0.874,7765.430,12.329
3,Random_Forest,0.880,7579.375,11.827
4,LGBM,0.882,7524.928,12.622
5,XGBoost,0.887,7347.550,11.934


In [82]:
df_select = df_scale[select_list]
df_select

,Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,Truyền động 1_駆動1,D/G_Ｄ／Ｇ,Rank_ランク,Loại xe_車種,Ngày bán hàng_売上日,Tháng bán hàng_売上月,Năm bán hàng_売上年
0,-0.365151,-1.450499,-0.352448,-0.514431,0.298227,-0.828100,-0.599671,-0.224836,-1.441442,-0.116579
1,0.062495,0.330100,-0.613234,-0.514431,0.298227,-0.828100,-0.599671,1.147770,-0.826008,-0.116579
2,0.437776,-0.840008,-0.465220,-0.514431,0.298227,0.591995,0.227718,0.274294,-1.441442,-0.116579
3,-0.365151,-1.450499,-0.352448,-0.514431,0.298227,0.591995,-0.599671,1.272552,0.404861,-0.116579
4,-0.696795,1.803031,-1.148903,-0.514431,0.298227,-0.828100,-0.599671,-0.848747,-1.749159,-0.116579
...,...,...,...,...,...,...,...,...,...,...
14921,-1.875004,-0.280391,0.993773,-0.514431,0.298227,-0.828100,2.709885,-1.223094,-0.826008,1.216542
14922,1.092337,1.621337,1.395525,-0.514431,0.298227,-0.828100,-0.599671,-1.597441,1.020295,-0.783140
14923,0.385412,1.190117,1.395525,-0.514431,0.298227,0.591995,-0.599671,-1.597441,1.020295,-0.783140
14924,1.293069,0.911520,0.542683,1.591549,0.298227,1.444052,1.055107,-1.597441,1.020295,-0.783140


In [83]:
x_train, x_test, y_train, y_test = train_test_split(df_select, df_target, test_size=0.2, random_state=42)

In [84]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[27612.57071169 68768.92613875 33461.86188047 ... 30025.2934718
 31060.046272   31060.32566467]
[33592.19436113 68639.1185567  33592.19436113 ... 30152.6774723
 30152.6774723  30152.6774723 ]
[24860. 68675. 44660. ... 29250. 39685. 23980.]
[24790.   66300.   34510.   ... 27600.   40255.   24568.88]
[25188.39278497 68913.00452716 44450.68724057 ... 28406.13286651
 40646.26065476 26595.20831056]
[24147.18  64198.047 60341.723 ... 27074.23  40515.746 24115.914]


In [85]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.713,11713.849,18.782
1,Decision_Tree,0.530,15008.163,24.637
2,Extra_Trees,0.864,8070.706,11.009
3,Random_Forest,0.873,7789.653,10.941
4,LGBM,0.840,8749.900,13.875
5,XGBoost,0.861,8166.759,12.543


# Phu tung ノーズカット（個別）

In [86]:
df_ノーズカット = df_fil[(df_fil['Tên phụ tùng_商品名']=='ノーズカット（個別）')]

In [87]:
cat_col =  list(df_ノーズカット.select_dtypes(include='object'))
df_cat  = df_ノーズカット[cat_col]
df_cat.isnull().sum()

Vị trí_ロケーション                      0
Nv sản xuất_生産担当者                  0
Số nhập kho_入庫番号                   0
Nguồn nhập hàng_仕入先                0
Mã loại phụ tùng_商品コード             0
Tên phụ tùng_商品名                   0
Số nhập kho(phụ tùng)_入庫番号 (部品)    0
Tên xe_車名                          0
Model xe_車輌型式                      0
Engine model_エンジン型式                0
Khách hàng_仕向先                     0
Truyền động 1_駆動1                  0
TM_TM                              0
Truyền động 2_駆動2                  0
Chi tiết_詳細                        0
Rank_ランク                           0
Loại xe_車種                         0
Nhà sản xuất_メーカー                  0
Chi tiết xếp hạng_ランク詳細            0
Số body_車体番号                       0
CONTAINER_CONTAINER                0
Tên khách hàng_売上先名                0
D/G_Ｄ／Ｇ                            0
Đang tải nguồn_積込元                 0
dtype: int64

In [88]:
num_col = list(df_ノーズカット.select_dtypes(include=['int64','float64']))
df_num = df_ノーズカット[num_col]
df_num.isnull().sum()

Ngày sản xuất_生産日                                       0
Tháng sản xuất_生産月                                      0
Năm sản xuất_生産年                                        0
Tổng giá nhập_仕入合計                                      0
Số phút công_標準工数                                       0
Kết quả_生産高                                             0
Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額    0
Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額     0
Số lượng bán ra_販売数量                                    0
Thành tích bán ra_販売実績                                  0
Số km đã chạy_走行距離                                      0
Năm sản xuất_年式                                         0
Số chỉ định loại_型式指定番号                                 0
Số phân loại danh mục_類別区分番号                            0
Đơn giá mua_仕入単価                                        0
Giờ làm việc_工数                                         0
Đơn giá bán hàng_売上単価                                   0
Sản lượng bán 

In [89]:
df_target = df_ノーズカット['Thành tích bán ra_販売実績']
df_target

0        12000.0
2        10000.0
3        12000.0
4        10000.0
5         9000.0
          ...   
20329    12000.0
20334    10000.0
20342    10000.0
20405    12000.0
20423    12000.0
Name: Thành tích bán ra_販売実績, Length: 3524, dtype: float64

In [90]:
le = LabelEncoder()
df_cat_encoded = df_cat.apply(lambda col: le.fit_transform(col))
df_encode = pd.merge(df_cat_encoded, df_num, on = df_cat_encoded.index, how='outer')
df_encode.drop(columns = ['key_0'], inplace=True)
df_encode

,Vị trí_ロケーション,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,...,Đơn giá bán hàng_売上単価,Sản lượng bán ra_売上金額,Ngày bán hàng_売上日,Tháng bán hàng_売上月,Năm bán hàng_売上年,Ngày mua_仕入日,Tháng mua hàng_仕入月,Năm mua_仕入年,Số phiếu_伝票番号,Mã nhà cung cấp_仕入先コード
0,0,12,862,782,0,0,2930,156,187,193,...,12000.0,12000,30,3,2016,22.0,12.0,2012.0,39332,990010.0
1,0,20,886,880,0,0,2660,118,37,86,...,10000.0,10000,19,2,2015,31.0,1.0,2015.0,24925,990000.0
2,0,30,1279,852,0,0,2688,166,514,117,...,12000.0,12000,23,2,2015,15.0,1.0,2015.0,24945,319800.0
3,0,5,1410,351,0,0,2710,172,1,242,...,10000.0,10000,24,3,2015,23.0,2.0,2015.0,25365,208750.0
4,0,33,1411,568,0,0,2718,114,167,94,...,9000.0,9000,26,5,2015,24.0,3.0,2015.0,26148,518170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3519,0,16,406,689,0,0,1270,136,341,5,...,0.0,0,30,11,2013,7.0,12.0,2011.0,19680,990010.0
3520,0,33,2369,607,0,0,3202,26,288,190,...,0.0,0,30,8,2017,27.0,2.0,2017.0,45183,990010.0
3521,0,20,2581,268,0,0,37,57,280,2,...,0.0,0,13,12,2017,6.0,12.0,2017.0,46320,312800.0
3522,0,29,364,854,0,0,1352,119,550,37,...,0.0,0,6,2,2014,24.0,2.0,2012.0,20406,208900.0


In [91]:
ss = StandardScaler()
df_scale = pd.DataFrame(ss.fit_transform(df_encode),columns = df_encode.columns)
df_scale.drop(columns=['Thành tích bán ra_販売実績','Đơn giá bán hàng_売上単価','Sản lượng bán ra_売上金額'],inplace=True)
x_train, x_test, y_train, y_test = train_test_split(df_scale, df_target, test_size=0.2, random_state=42)

In [93]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[11592.80629434 12345.54949086  8343.63554402 10083.50739627
  9144.06539202  9574.84297148 10040.04397741 10714.65378655
 11044.19003343 11390.79014404 10322.06001415 13501.91707448
 10382.11707206 10024.77319637 11903.15893426 10392.61047788
 10921.50673037 11845.35927502 10158.27068579  9664.31563815
 10773.01363959  9360.25097972 12303.8885052   9614.36668729
  9929.02035666 12360.73724534  8813.2135406  12146.36273689
 11595.97051788 10194.01654765 11907.49068362  9307.55662898
 11751.9657983  12673.44362838 10059.43634312  9391.92188902
 10247.97931839 11608.29463971 12051.34137605  9092.02925438
  8347.23982569 12901.19337919  9855.50235288  9863.03049015
 11541.65064277 11792.12997929 10486.58902165 11947.72575578
  7877.13515757 10087.64819504 12010.09560813 12251.0605416
 10463.8892219  10649.31027949 10590.30259621  9819.09455508
  9487.88049565  7851.61098913 16789.68687759 11028.4756528
 10961.1449955  10941.81408305  9837.92786531  9675.91250351
 11036.06641677  8890.5083

[12060. 12540.  9410. 10000.  8890.  9730. 10380. 14250. 10850. 11380.
 10980. 13820. 11270. 10520. 11470. 10790. 10820. 12000. 10510. 10110.
 11380.  9690. 12000.  9650. 11310. 12740.  8360. 12060. 11840.  8910.
 12335. 10840. 12040. 12220. 10640.  9990. 10240. 11300. 12790.  9040.
  7745. 12190. 10000. 10040. 12070. 10290. 11450. 12700.  7340.  9940.
 12470. 13050. 11720.  9810. 10100.  9950. 10010.  7865. 12490. 10620.
 10770. 10830.  9960.  9980. 10860.  7540. 10000. 11130. 15590. 10580.
  8950.  9680.  9620.  9950. 11070. 11050.  9580.  8870. 12100.  7030.
 11020. 10600. 10090. 11680. 12160. 10510.  9570. 11260.  9600.  8765.
 11140.  9420. 13130. 10640. 12030. 12190. 11160.  9620.  9950. 11250.
 10120. 10390.  7855. 10570. 12030. 10900. 11600. 11370. 10880. 10000.
  7340. 10180. 13330. 12580. 11270. 11410. 12890. 11670. 17130. 13020.
  8720. 10720. 13370. 10080.  8000. 11710. 10190. 10800. 10990. 11350.
 13530. 11680. 10300. 10460.  7980. 10750.  8850. 10070. 11060. 11360.
 11060

[11793.569  15789.754   8135.849   9567.094   7536.6147  9224.396
 11170.342  13773.967  12291.994  11566.312  10623.497  13554.234
  9920.119  10593.188  11603.417   9890.767  11634.701  11994.098
 10498.825   9352.622  10827.378   8331.64   12939.223   9500.439
  9848.059  13322.035   8040.6353 11514.667  12348.382   8525.007
 12214.309   9499.68   11453.237  13351.221  10372.347  10041.632
  9834.861  12305.924  12793.113   8790.123   8184.4907 12562.618
 10088.081  10128.141  12606.348  11033.919  10306.404  12459.6455
  7743.975  10285.765  12882.496  12861.711  10600.616  10687.523
  9747.304   9423.785   9935.131   8621.528  12356.206  11354.806
 10923.682  12116.537   9840.818   9939.626  11623.039   7743.9644
  9943.052   9987.778  13931.807   9779.261   8928.413   9565.46
  7869.211   9933.162  12030.634  10690.533   8832.043  10893.423
 10269.202   7202.7314 13928.563  10130.249   9666.819  12654.178
 12532.574  11148.374  10008.337  10684.7     9108.551   9136.097
 11446.12

In [94]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.372,1915.717,11.004
1,Decision_Tree,0.289,2037.668,12.835
2,Extra_Trees,0.528,1660.052,8.828
3,Random_Forest,0.334,1973.212,9.037
4,LGBM,0.464,1769.187,9.803
5,XGBoost,-0.848,3286.358,10.758


In [95]:
df_shap = df_scale[shap_list]
df_shap

,Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額,Mã loại phụ tùng_商品コード,Tổng giá nhập_仕入合計,Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額,Tên khách hàng_売上先名,Engine model_エンジン型式,Số phút công_標準工数,Số nhập kho_入庫番号,Khách hàng_仕向先
0,0.015365,0.0,-0.836068,-0.212885,0.993177,1.072949,0.730873,-0.884211,0.046482
1,0.015365,0.0,1.136016,4.487666,1.203859,-0.281394,0.730873,-0.860619,0.915461
2,0.015365,0.0,-0.516591,-0.212885,0.887836,0.110986,0.730873,-0.474299,0.752527
3,0.015365,0.0,-0.516591,-0.212885,0.887836,1.693163,0.730873,-0.345525,0.752527
4,0.015365,0.0,-0.536558,-0.212885,0.887836,-0.180135,0.730873,-0.344542,0.752527
...,...,...,...,...,...,...,...,...,...
3519,0.015365,0.0,-0.443377,-0.212885,-1.008303,-1.306645,0.730873,-1.332460,-1.202675
3520,-0.090775,0.0,-0.836068,-0.212885,-0.481598,1.034977,-1.368227,0.597174,-0.822497
3521,-0.090775,0.0,0.415214,-0.212885,-1.008303,-1.344617,-1.368227,0.805571,-1.202675
3522,0.015365,0.0,-0.361289,-0.212885,-1.008303,-0.901607,0.730873,-1.373746,-1.202675


In [96]:
x_train, x_test, y_train, y_test = train_test_split(df_shap, df_target, test_size=0.2, random_state=42)

In [97]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[12162.26238215 11460.60187243  9267.23010754 15044.18475338
  9924.5135087   9741.15806934 10454.62568998  9706.12698024
 11151.95860919 10980.30421079 10822.43508751 13216.01439558
 11179.00985099 10609.53543762 11498.24503992 10000.91610335
 11461.49629327 12098.74066141 10209.80540313 10053.95488216
 10585.12906132  9644.85782026 12233.27696098  9424.84124762
  8872.72465384 11926.34425887  9654.36146865 11217.60120935
 10832.59026118  9931.84821626 11998.68858462 10082.6748272
 11278.26108139 12299.65662669 13522.58588223  9700.75185392
 10732.87041553 11303.90446709 12389.18977586  9736.86061094
  9653.44448947 12453.76263004  9441.73822034  9806.49903438
 12198.17863809 10332.34156341 10907.89688078 15892.40829036
  8238.94335611 10186.76235165 11196.70851568 11927.07773134
 10451.86062904 10334.17132634 10710.71738764  9733.38053103
  8875.28900766  8519.41299307 13358.81378641  9266.03392196
 10686.9767122  10469.83040832  9654.36146865  9573.39370887
 10696.06737594  8190.788

[14020. 11960.  8650. 10400.  8200.  9500. 10060. 13100. 10130. 11060.
 12300. 14620. 10130. 10820. 11080.  9710. 11270. 12000. 11050. 10680.
 10300. 10640. 12590. 10030. 10260. 12340.  8880. 12000. 11520. 11350.
 12130. 11900. 13160. 12220. 11800. 10000. 12340. 10220. 12550.  8910.
  7930. 11950.  9570. 10000. 13250. 10220. 12320. 14570.  7555.  9970.
 11690. 12670. 10000.  9900.  9970.  9990. 10000.  8015. 12330. 10860.
 10230. 11820. 10000. 10000. 10470.  7920. 10000. 10200. 17720. 10200.
  8960.  9300.  7970. 10100. 10030. 10220.  9710.  9070. 13170.  7710.
 11280. 10000. 11100. 13530. 12020. 10240. 12320. 11920. 10200.  9030.
 12370. 10040. 10230. 11480.  9590. 12020. 11240.  8010.  9740. 11830.
 10060.  9930.  7465. 11210. 11980. 10000. 10390. 12920. 10760. 10000.
  7950. 12030. 12000. 15140.  9940. 10450. 11710. 10380. 16440. 12580.
  9530. 11730. 12040. 10040.  8020. 10000. 11780. 10870. 13250.  9680.
 13660. 11480. 10670.  9890.  8210. 10710.  6800.  9690. 10080. 12000.
 10090

[13237.747  11413.535   8664.175  16731.955   9471.199   9757.853
 10271.495   9649.625  10721.194  11008.572  11529.616  11035.353
 11154.784  10647.911  11639.206   9915.196  11432.275  11969.151
 10311.048   9765.181  10597.158  12005.156  12049.205   9843.414
  9816.266  11458.938   8701.478  11694.193   9731.475   9994.799
 12219.595  11970.758  13816.79   11950.026  11885.879   9846.603
 11880.951  11471.952  12648.245   8850.479   8585.304  13580.159
  8493.907  10185.204  12832.279   9961.766  11985.942  15427.162
  7754.7373 10543.624  11487.448  13791.328  10722.743   9691.47
  9499.349   9686.142   9569.067   8024.6475 12083.344   8255.517
 10377.053  10957.676  10081.561   9636.03   10091.918   8396.059
  9962.708   9345.222  14649.617   9459.218   8393.859   6808.412
  7953.214  11585.296  11214.608  11444.326  10023.7     9542.98
 11788.724   7705.0674 10095.429   9670.234   9969.45   11359.225
 12035.523  10257.386  10907.158  11074.749  10239.982  10408.638
  9895.099  

In [98]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,-0.097,2532.510,11.817
1,Decision_Tree,-0.350,2808.796,13.679
2,Extra_Trees,0.047,2360.424,10.297
3,Random_Forest,-0.380,2839.944,11.328
4,LGBM,0.242,2104.105,11.625
5,XGBoost,-0.614,3071.583,11.670


In [99]:
select_list = ['Tên xe_車名','Model xe_車輌型式' , 
               'Engine model_エンジン型式','Truyền động 1_駆動1', 'D/G_Ｄ／Ｇ', 'Rank_ランク', 'Loại xe_車種',
              'Ngày bán hàng_売上日','Tháng bán hàng_売上月','Năm bán hàng_売上年']

In [101]:
df_select = df_scale[select_list]
x_train, x_test, y_train, y_test = train_test_split(df_select, df_target, test_size=0.2, random_state=42)

In [102]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[11824.95124083 10680.47788181  9788.29675582 10939.10684062
  9716.05578393 10300.63638984 10229.35300694 10368.79799946
 11056.55333889 11171.23535811 10339.85585316 10826.48351831
 11059.0550858  10632.41963322 10047.91627575 10117.39068143
  9959.68491688 11200.15366476 10614.62752789  9819.0615996
 10605.70588172  9676.54330034 12233.7326749  10214.30699995
 10701.49530095 12018.50931007  9411.88842538 10724.97603502
 11315.4214087  10235.85048328 11055.17214065 10428.73922206
 11207.04062962 13207.95725883 10911.92233869 10100.22797363
 11892.19583227 11519.81105859 11347.54690022  9601.55491662
 11053.98367683 11014.6453545  10326.61586971 10262.01306681
 10640.89415856 10494.75772548 10511.65938708 11222.56145208
 10751.24552924 10146.70996291 11123.71945896 11307.18736208
 11574.19286085 10577.56659181 11133.5966757   9822.03451631
 10366.18014812  9268.17512291 19492.75847617 11118.14698588
 10590.65231399 11000.71653291 10204.60832053 10092.26128694
 10493.27529853 10602.532

[11850.  9990.  9850. 11490.  8390.  9980. 11900. 15000. 10190. 11560.
 11170. 10650. 10330. 10780. 10430. 11450. 11130. 12000. 11660.  9560.
 11190.  9300. 11720.  9430. 10250. 12290.  8700. 11390. 11860.  9020.
 11240. 11750. 12830. 11600. 10270.  9840. 13730. 11630. 11610.  9000.
  9495. 11320.  9940. 10320. 11490. 10030. 10100. 12070.  8130.  9910.
 11940.  8915. 10900. 10730. 10360.  9950. 10110.  8040. 12040. 11400.
 10230. 12440. 10280. 10030. 10660.  8330. 10090. 10140. 16220.  9670.
  9000. 10080. 10040.  9710. 10950. 10200.  9660.  9320. 10820.  7360.
 11590. 11200.  9680. 11810. 11990. 10340.  9880. 10140.  9380.  8965.
 10280.  8940. 12880. 11160. 11530. 12340. 11090.  9580.  9950. 12145.
  9980. 10190.  9940. 10440. 11960. 10110. 11980. 12700. 11550. 10310.
  7380. 10230. 11940. 11420. 10630. 10820. 13360. 13560. 17000. 11680.
 10340.  8515. 11330.  9500.  8100. 10210.  9640. 10040. 12790.  9080.
 12730. 10680. 10020. 14900.  9270.  9310.  8100. 10100. 11710. 12540.
 10620

[12220.438  11200.798   9964.033  14721.938   8738.852   9808.239
 10909.78   13792.305  10190.563  11735.313   9256.5625  9581.513
 10233.341   9524.314  10377.535  11005.359  10759.972  11850.343
 10934.73    9837.728  11341.747   8739.923  10690.804   9651.887
 10386.6    12708.367   8718.637  10864.481  10732.748   8350.539
 10873.511  10674.099  11118.821  12582.896  10421.495   9634.2
 10168.495  11882.293  12007.089   8853.149  10287.971  11640.348
 10468.004  10096.693  10676.8125  9699.367  11580.293  10398.3545
  8557.361   9571.918  10997.118   9899.223  10006.659   9288.508
  9807.598   9719.514  10074.227   7598.378  38557.375  10840.204
 10262.723  11949.713  10434.917   9887.739   9485.239   8828.585
 10893.279  10159.967  13356.6045  9880.906   8910.913   9530.656
  7353.1216  9638.708  11656.088  11635.797   9796.662   8992.523
 11498.729   8227.056  11680.459  10476.659   8719.637  12992.072
 10860.244   8316.6875  9960.738  10010.043  10054.234   8980.235
  9743.563 

In [103]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.124,2262.668,14.016
1,Decision_Tree,-0.229,2679.443,14.465
2,Extra_Trees,0.288,2040.043,11.447
3,Random_Forest,0.238,2110.520,11.461
4,LGBM,0.287,2040.606,12.646
5,XGBoost,0.261,2077.466,11.269
